In [8]:
import json
import datetime
import numpy as np
import matplotlib.pyplot as plt
import dateparser
import TwitterSearch as tw
import emoji as emo

from ribbonline.tweet import Tweet
from ribbonline.collection import TweetCollection
from ribbonline.opinion import DailyOpinionSplit
%autoreload 2

In [3]:
credentials = rb.util.load_credentials()

ts = tw.TwitterSearch(
    consumer_key = credentials['api_key'],
    consumer_secret = credentials['api_secret_key'],
    access_token = credentials['access_token'],
    access_token_secret = credentials['access_token_secret']
)

AttributeError: module 'ribbonline' has no attribute 'util'

In [ ]:
since = datetime.date(2018,11,29)
until = datetime.date(2018,11,30)
tso = tw.TwitterSearchOrder()
tso.set_keywords(['#Catalunya', '#Cataluña'], or_operator=True)
tso.set_include_entities(False)
tso.set_since(since)
tso.set_until(until)

In [ ]:
tweets = TweetCollection([])
for tweet_data in ts.search_tweets_iterable(tso):
    tweets.append(Tweet(tweet_data))

In [ ]:
daily_opinion = DailyOpinionSplit(date=since, tweet_collection=tweets)
daily_opinion

# Stuff

In [ ]:
tuo = tw.TwitterUserOrder('Miotroyo2parte') # create a TwitterUserOrder
# start asking Twitter about the timeline
for tweet_data in ts.search_tweets_iterable(tuo):
    flagtweet = Tweet(tweet_data)
    break

In [ ]:
flagtweet.username_has_flag